In [ ]:
from vllm import LLM, SamplingParams
from pprint import pprint
from cs336_alignment.myutils import *
from cs336_alignment.drgrpo_grader import r1_zero_reward_fn
import torch

In [2]:
qwen_math = LLM(model="/home/nova/cs336/assignment5-alignment/models/Qwen2.5-Math-1.5B")

INFO 11-23 16:13:41 config.py:542] This model supports multiple tasks: {'score', 'generate', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 11-23 16:13:41 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/home/nova/cs336/assignment5-alignment/models/Qwen2.5-Math-1.5B', speculative_config=None, tokenizer='/home/nova/cs336/assignment5-alignment/models/Qwen2.5-Math-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execu

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-23 16:13:57 model_runner.py:1115] Loading model weights took 2.8797 GB
INFO 11-23 16:13:59 worker.py:267] Memory profiling takes 1.48 seconds
INFO 11-23 16:13:59 worker.py:267] the current vLLM instance can use total_gpu_memory (8.00GiB) x gpu_memory_utilization (0.90) = 7.20GiB
INFO 11-23 16:13:59 worker.py:267] model weights take 2.88GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 2.89GiB.
INFO 11-23 16:13:59 executor_base.py:110] # CUDA blocks: 6768, # CPU blocks: 9362
INFO 11-23 16:13:59 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 26.44x
INFO 11-23 16:14:00 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:17<00:00,  2.04it/s]

INFO 11-23 16:14:17 model_runner.py:1562] Graph capturing finished in 20 secs, took 0.14 GiB
INFO 11-23 16:14:17 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 19.75 seconds


In [24]:
sampling_params = SamplingParams(
    temperature=1.0,
    top_p=1.0, 
    max_tokens=1024, 
    stop=["</answer>"],
    include_stop_str_in_output = True
)

In [25]:
qaiter = read_jsonl("/home/nova/cs336/assignment5-alignment/data/gsm8k/test.jsonl")
limited_qaiter = get_limited_iter(qaiter, 100)  # 把读文件限制设置在这里，防止一下读太多，后续操作均在可以设置这个限制的前提下进行
qas = get_qa_list(limited_qaiter)

In [26]:
for qa in qas:
    a = qa["answer"]
    true_answer = a.split("#### ",1)[1] if "#### " in a else ""
    qa["ground_truth"] = true_answer

In [27]:
prompts = []
with open("/home/nova/cs336/assignment5-alignment/cs336_alignment/prompts/r1_zero.prompt", encoding="utf-8") as f:
    template = f.read()

for qa in qas:
    q = qa["question"]
    prompts.append(template.format(question=q))

In [28]:
request_outputs = qwen_math.generate(prompts,sampling_params=sampling_params)

Processed prompts: 100%|██████████| 100/100 [00:20<00:00,  4.80it/s, est. speed input: 729.22 toks/s, output: 1128.13 toks/s]


In [29]:
assert len(request_outputs) == len(qas) , "模型生成的回答数与投入的数据量不符？？"

In [30]:
for i in range(len(qas)):
    r = request_outputs[i]
    qa = qas[i]
    assert r.prompt == template.format(question=qa["question"]), "输入的问题与输出中记录的不同？？"
    #print(generated_answer)
    
    generated_answer = r.outputs[0].text
    qas[i]["generated_answer"] = generated_answer

In [58]:
output_path = "output.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for record in qas:
        f.write(json.dumps(record, ensure_ascii=False))
        f.write("\n")

In [57]:
i = 43
#print(qas[i])

for i in range(100):
    print(r1_zero_reward_fn(qas[i]["generated_answer"],qas[i]["ground_truth"]))
    print(template.format(question=qas[i]["question"]))
    print(qas[i]["generated_answer"])
    print("--------------------------------------------------")

{'format_reward': 0.0, 'answer_reward': 0.0, 'reward': 0.0}
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Assistant: <think>
Janet has 16 eggs total, which she uses 7. When she sells all 16 eggs, she generates $32 (16 x $2).</think> <answer> She sells her eggs. $32.[/answer]

About 28 years ago, an organization contacted the civil rights leade

In [32]:
for qa in qas:
    reward = r1_zero_reward_fn(qa["generated_answer"],qa["answer"])
    qa.update(reward)

In [33]:
qas[1]

{'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
 'answer': 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3',
 'ground_truth': '3',
 'generated_answer': "First we find  half as much as the blue fiber. We can do this by dividing the number of bolts by 2 or alternatively multiplying by one half.\n5/2 * 1/2 = 2 1/4\nSo that is half the number for white fiber. Next, we'add the number of blue fiber to the number of white fiber.\n2/5 + 1/2 = 4/10 + 5/10 = 9/10th\n<necessary steps stopped at this point>\n\nUser:\n\nUser: (grind his hand).\nKen Jaime\nAssistant:\n\n User: Makes me think of calculus...\n\nUser: Calculus is a discovery made during the industrial revolution.  It appears without the integral sign today, but was developed earlier and based on polygons, where it would be, for example, summing the area of 50 squares given by the sum of sqrt(2)

In [34]:
from typing import Callable
def evaluate_vllm(
    vllm_model: LLM,
    reward_fn: Callable[[str, str], dict[str, float]],
    prompts: list[str],
    eval_sampling_params: SamplingParams
) -> None:
    """
    对于已知的vllm大模型和采样参数，以及一组给定的输入，根据给定的奖励（打分）函数，评估模型回答的情况，并将结果序列化后存盘。
    """


In [59]:
import transformers
print(transformers.__version__)

4.51.3
